In [44]:
import numpy as np
import pandas as pd
import csv
import os
import subprocess

In [ ]:
matricula = 82834#num da pra usar o 00
num_var = 9#usei 9 pra ter 10 campos com o de resultado ja que nao tem a quantidade no slide,o gerador usa entre 5 e 10 tbm
precisao = 4#igual no exemplo dado
arquivos_gerados = []
for i in range(10):
    seed = matricula + (i * 100)
    nome_arquivo = f"{seed}.csv"
    cmd = f"python gerador.py -v {num_var} -p {precisao} -s {seed} -a {nome_arquivo}"
    subprocess.run(cmd, shell=True)
    if os.path.exists(nome_arquivo):
        arquivos_gerados.append(nome_arquivo)
    else:
        print(f"Erro ao gerar {nome_arquivo}")

In [49]:
def ler_sistema(arquivo):
    dados = []
    with open(arquivo, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            dados.append([float(x) for x in row])
    matriz_estendida = np.array(dados)
    n = len(matriz_estendida)
    A = matriz_estendida[:, :n]
    b = matriz_estendida[:, n]
    return A, b
def analisar_convergencia(A, metodo='jacobi'):
    D = np.diag(np.diag(A))
    L = np.tril(A) - D
    U = np.triu(A) - D
    if metodo == 'jacobi':
        T = -np.linalg.inv(D).dot(L + U)
    else: # gauss
        T = -np.linalg.inv(L + D).dot(U)
    av, _ = np.linalg.eig(T)
    raio_espectral = max(abs(av))
    return T, raio_espectral
def resolver_sistema(A, b, metodo='jacobi', tol=1e-5, max_iter=50):
    n = len(b)
    x = np.zeros(n, dtype='double') 
    historico = []
    historico.append([0, str(x), "-"])
    msg_convergencia = ""
    T, rho = analisar_convergencia(A, metodo)
    #print("Matriz de iteração:")
    #print(T)
    if rho >= 1:
        msg_convergencia = f"O raio espectral ({rho:.4f}) >= 1, pode não acabar convergindo."
    else:
        msg_convergencia = f"O sistema vai convergir, raio espectral: {rho:.4f}"
    for k in range(1, max_iter + 1):
        xant = np.copy(x)
        if metodo == 'jacobi':
            x_novo = np.zeros(n, dtype='double')
            for i in range(n):
                soma = sum(A[i, j] * xant[j] for j in range(n) if j != i)
                x_novo[i] = (b[i] - soma) / A[i, i]
            x = x_novo
        elif metodo == 'gauss-seidel':
            for i in range(n):
                soma = sum(A[i, j] * x[j] for j in range(n) if j != i)
                x[i] = (b[i] - soma) / A[i, i]
        erro = np.linalg.norm(x - xant, np.inf)
        x_str = str(np.round(x, 8))
        historico.append([k, x_str, f"{erro:.8f}"])
        if erro < tol:
            break
    df = pd.DataFrame(historico, columns=['Iteração', 'Aproximação x', 'Erro'])
    return df, msg_convergencia, x

In [50]:
for arquivo in arquivos_gerados:
    print("================================================================================")
    print(f"Arquivo: {arquivo}") 
    A, b = ler_sistema(arquivo)
    #JACOBI
    print("Método de Jacobi")
    df_j, msg_j, sol_j = resolver_sistema(A, b, 'jacobi')
    print(msg_j)
    display(df_j)
    print(f"Solução aproximada encontrada\nx = {sol_j}")
    #GAUSS
    print("\nMétodo de Gauss-Seidel")
    df_g, msg_g, sol_g = resolver_sistema(A, b, 'gauss-seidel')
    print(msg_g)
    display(df_g)
    print(f"Solução aproximada encontrada\nx = {sol_g}")

Arquivo: 82834.csv
Método de Jacobi
O sistema vai convergir, raio espectral: 0.3602


,Iteração,Aproximação x,Erro
0,0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],-
1,1,[ -5.38625584 9.2618244 2.15035196 -4.57...,12.25574286
2,2,[-2.83922181 8.69230216 0.98205093 -7.504494...,4.19336563
3,3,[ -2.80745545 9.36584797 1.01266699 -6.36...,1.60453334
4,4,[-3.04651299 9.00339743 0.9317692 -6.219414...,0.42986925
5,5,[-3.04969517 9.12961267 1.04705752 -6.216966...,0.13764463
6,6,[-3.02478344 9.0790197 1.00349373 -6.282953...,0.06598677
7,7,[-3.02940587 9.09655272 1.01063813 -6.262943...,0.02073711
8,8,[-3.0308306 9.09044675 1.00941595 -6.262728...,0.00726891
9,9,[-3.03083456 9.09232813 1.01068859 -6.262724...,0.00244584


Solução aproximada encontrada
x = [-3.03060909  9.09183504  1.01019938 -6.26326549  5.05102056  0.20203451
  2.62652526 -2.00000535 -9.90000063]

Método de Gauss-Seidel
O sistema vai convergir, raio espectral: 0.1358


,Iteração,Aproximação x,Erro
0,0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],-
1,1,[-5.38625584 9.68119347 1.84568428 -4.312852...,9.68119347
2,2,[-2.92062198 8.73858115 0.54574043 -7.086416...,2.83718616
3,3,[-2.96202563 9.36068769 1.03616023 -6.252197...,0.83421883
4,4,[-2.99629867 9.08184481 1.0114869 -6.254126...,0.27884288
5,5,[-3.03267757 9.09063508 1.01050755 -6.265024...,0.03637889
6,6,[-3.03057207 9.09192894 1.01030224 -6.262900...,0.00212415
7,7,[-3.03064261 9.09175441 1.01018575 -6.263298...,0.00039828
8,8,[-3.03061554 9.09184207 1.01019914 -6.263265...,0.00008767
9,9,[-3.03060818 9.09183356 1.01019877 -6.263265...,0.00000852


Solução aproximada encontrada
x = [-3.03060818  9.09183356  1.01019877 -6.26326568  5.05101996  0.20203533
  2.62652505 -2.00000675 -9.89999948]
Arquivo: 82934.csv
Método de Jacobi
O sistema vai convergir, raio espectral: 0.3719


...

Solução aproximada encontrada
x = [ 4.24285862 -7.00000135 -4.64694006  9.8999964  -2.62653304 -0.6061125
  6.26325374 -7.4755075   1.8183795 ]
Arquivo: 83734.csv
Método de Jacobi
O sistema vai convergir, raio espectral: 0.3729


,Iteração,Aproximação x,Erro
0,0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],-
1,1,[ 4.89557043 5.00407019 3.14739133 -1.879464...,8.80542901
2,2,[ 4.09654726 4.74256693 4.033245 -1.191206...,1.16832401
3,3,[ 4.37164931 4.81426069 4.16987671 -1.785763...,0.59455695
4,4,[ 4.20792867 4.65147057 4.03718922 -1.858010...,0.19860448
5,5,[ 4.21537522 4.64920826 4.00024292 -1.815528...,0.08221326
6,6,[ 4.25150673 4.65431441 3.99883953 -1.823648...,0.03613151
7,7,[ 4.24304104 4.64628698 4.00011997 -1.819564...,0.00847657
8,8,[ 4.24188567 4.64666934 4.00028144 -1.817733...,0.00339895
9,9,[ 4.24320255 4.64712177 3.99997381 -1.818614...,0.00131688


Solução aproximada encontrada
x = [ 4.24284568  4.64694872  3.99998794 -1.81834684  3.43470848  9.99999214
  7.87958087 -4.24287275  0.60611182]

Método de Gauss-Seidel
O sistema vai convergir, raio espectral: 0.2799


,Iteração,Aproximação x,Erro
0,0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],-
1,1,[ 4.89557043 4.08733455 2.85042815 -2.478770...,8.20003964
2,2,[ 3.97983513 4.40618609 3.73043874 -1.652040...,1.88836118
3,3,[ 4.36729952 4.68157807 4.03325834 -1.836267...,0.38746439
4,4,[ 4.2150888 4.63706067 3.99253142 -1.808768...,0.15221072
5,5,[ 4.25100782 4.64967678 4.00212048 -1.820587...,0.03591902
6,6,[ 4.24058693 4.64614506 3.99935873 -1.817675...,0.01042089
7,7,[ 4.2434863 4.64716671 4.00015925 -1.818530...,0.00289938
8,8,[ 4.2426696 4.64688698 3.99993959 -1.818296...,0.00081670
9,9,[ 4.24289722 4.64696595 4.00000132 -1.818363...,0.00022762


Solução aproximada encontrada
x = [ 4.24284624  4.64694837  3.99998754 -1.81834844  3.43471083  9.9999906
  7.87958115 -4.24287465  0.60611136]
